In [41]:
# Zener Diode Voltage Regulator

import math
import matplotlib.pyplot as plt
import numpy as np


<h2>When the Load Resistor R_L Disconnected</h2>

<h3>Current Calculations</h3>
<ul>
    <li><p>When the load resistor is connected, the current through the load resistor should remain 10 mA,
    and the current through the Zener diode should ideally be 5 mA, but can conduct at a current as low as 1 mA and as high as 9 mA.</p></li>
    <li><p>When the load resistor is not connected, the 10 mA that would otherwise flow through the load resistor would increase the current through
    the Zener diode by 10 mA. Therefore, the maximum current through the Zener diode when the load resistor is not connected is 19 mA.</p></li>
    <li><p>A maximum current through the Zener diode of 19 mA is still less than the maximum specified in the data sheet of 20 mA.</p></li>
</ul>
<h3>Power Calculations</h3>
<ul>
    <li><p>Moreover, if the current through the Zener diode is 19 mA, the power dissipated by the Zener diode would then be calculated as P_max = I * V.</p></li>
    <li><p>P_max = 19 mA * 10 V = 190 mW. 190 mW is less than the maximum specfified power of 250 mW.</p></li>
</ul>

In [42]:
# Choosing R1 and C values from current calculations

# Constants
R_L = 1000  # [Ohms]
f = 60      # [Hz]
T = 1 / f   # [s]
V_D = 0.8   # [V] voltage of diodes in the wheatstone bridge

# Conditions for voltage of Zener diode at I_ZK = 5 mA
V_z = 10
V_z_min = 9.4
V_z_max = 10.6

# Zener Diode Currents
I_Z0 = 5 * 10 ** -3             # Ideal given operating current:   5 mA
I_ZT2 = 1 * 10 ** -3            # Minimum given operating current: 1 mA 
I_ZT3 = 20 * 10 ** -3           # Maximum given operating current: 20 mA
deltaI = abs(I_Z0 - I_ZT2)      # Calculated deviation with s.f. +- 4 mA from ideal 
I_Z_Min = I_ZT2                 # Minimum calculated IZ current = 1 mA
I_Z_Max = I_Z0 + deltaI         # Maximum calculated w s.f. IZ = 9 mA

# Current Calculations
I_L = V_z / R_L         # 10 mA
I_Z_ideal = I_Z0        # 5 mA
I_in = I_Z_ideal + I_L  # 15 mA

# ------------ Calculating R1 as a function of V_in_mid ------------------------
# Generally R1 = (V_in_mid - V_z) / I_in

# Conditions for V_in [V]
V_in_pk = 15.4                  # V_in_pk: 15.4 V
deltaV_in = 1.136               # Calculated deviation deltaV_in: 1.136 V
V_in_max = 15.4 + deltaV_in     # V_in_max: 16.536 V
V_in_min = 15.4 - deltaV_in     # V_in_min: 14.234 V


In [43]:
# Plotting Voltage Across Capacitor as a Function of Time Using Integration (Method 2 in lab sheet)

# Calculating R1 when V_in is at a maximum
R1_small = (V_in_max - V_z) / I_in

# Calculting R_TH when I_Z = 5 mA, I_L = 10 mA, I_in = 10 mA. Valid for V_Z in range of [9.4, 10.6] V.
R_Z = 20  # r_z = 20 Ohms @ I_Z = 5 mA

# R_TH = R1 + (R_Z||R_L)
R_TH = R1_small + ((R_Z * R_L) / (R_Z + R_L))

# Calculating C for smaller R1 value
term1 = -1 / (120 * R_TH)
term2 = 1 / (np.log(1 - ( ( (2 * deltaV_in) / (V_in_pk - 2*V_D - V_z) ) *(R1_small / R_TH) )))
C_1 = term1 * term2
C_1_print = C_1 * 10**6

print(f"\nC = {C_1_print:0.3f} uF when R1 = {R1_small:0.3f} Ohms as calculated from V_in_max = {V_in_max:0.3f} V")



C = 21.557 uF when R1 = 435.733 Ohms as calculated from V_in_max = 16.536 V


In [44]:
# Calculating C for larger R1 value

# Calculate larger R1
R1_large = (V_in_min - V_z) / I_in

# Calculate C from larger R1
term11 = -1 / (120 * R_TH)
term22 = 1 / (np.log(1 - ( ( (2 * deltaV_in) / (V_in_pk - 2*V_D - V_z) ) * (R1_large / R_TH) )))
C_2 = term11 * term22
C_2_print = C_2 * 10**6

print(f"\nC = {C_2_print:0.3f} uF when R1 = {R1_large:0.3f} Ohms as calculated from V_in_min = {V_in_min:0.3f} V")


C = 39.170 uF when R1 = 284.267 Ohms as calculated from V_in_min = 14.264 V


In [49]:
# Requirement Verifications for Currents
# Current across the Zener diode must be in range 1 mA < 9 mA when load resistor is connected

# Calculate current across the capacitor
def calculate_current_across_capacitor(V_in_pk, V_D, V_z, R1, tau, t):
    I_C = ((V_in_pk - 2 * V_D - V_z) / R1_small) * np.exp(-t / tau)  # [A]
    return I_C

def calculate_current_across_capacitor(t, I_C, C):
    Q_cumulative = np.zeros(len(t))
    V_C = np.zeros(len(t))

    for i in range(1, len(t)):
        delta_t = t[i] - t[i-1]
        avg_I = (I_C[i] + I_C[i-1]) / 2
        delta_Q = avg_I * delta_t
        Q_cumulative[i] = Q_cumulative[i-1] + delta_Q

# Calculate I_C(t)
t = np.linspace(0, 1/120, 1000)  # Create array of 50 points from 0 to t = 1/120 s
tau1 = R_TH * C_1
tau2 = R_TH * C_2
exponent_1 = -t / (tau1)
exponent_2 = -t / (tau2)

First I_C_1 value = 8.720930232558139 mA
Last I_C_1 value = 3.7312650816693433 mA

First I_C_2 value = 13.367729831144462 mA
Last I_C_2 value = 5.719406325935934 mA
